In [1]:
import pandas as pd

DATA = '/kaggle/input/adult-census-income-dataset/adult.csv'

df = pd.read_csv(filepath_or_buffer=DATA)
df['target'] = df['income'] == '>50K'
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income,target
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K,False
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K,False
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K,False
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K,False
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K,False


In [2]:
df['target'].value_counts().to_dict(), df['target'].value_counts(normalize=True).to_dict()

({False: 24720, True: 7841},
 {False: 0.7591904425539756, True: 0.2408095574460244})

Our classes are unbalanced; roughly 25% of the target variable is true.